**DEPRECATED** - Enter at your own risk

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from optiver import Directories
from optiver.bench import rmspe

dirs = Directories("../..")

In [2]:
def decision_split_df(train_features, train_targets, split):
    split_feature, split_value = split
    
    left_bool_index = train_features[split_feature] <= split_value
    right_bool_index = train_features[split_feature] > split_value
    
    train_features_left, train_targets_left = train_features[left_bool_index], train_targets[left_bool_index]
    train_features_right, train_targets_right = train_features[right_bool_index], train_targets[right_bool_index]
    
    return (train_features_left, train_targets_left), (train_features_right, train_targets_right)


def rss_from_mean(y):
    return ((y.mean() - y)**2).sum()


class DecisionTree:
    def __init__(self, *, value=None, split=None, left_tree=None, right_tree=None, annotations=None):
        self.value = None
        if value is not None:
            self.value = value
            self.is_node = True
        else:
            self.split = split
            self.left_tree = left_tree
            self.right_tree = right_tree
            self.is_node = False
        
        self.annotations = annotations or {}
        
    def clone(self):
        if self.is_node:
            tree = DecisionTree(value=self.value, annotations=self.annotations.copy())
        else:
            tree = DecisionTree(
                split=self.split, left_tree=self.left_tree.clone(), right_tree=self.right_tree.clone(), annotations=self.annotations.copy()
            )
        
        return tree
    
    def prune_best_cost(self, train_features, train_targets):
        trees, means, costs = zip(*list(self.annotations_and_means(train_features, train_targets)))
        
        best_index = np.argmax(np.array(costs))
        
        if costs[best_index] < 0.:
            return False
        
        trees[best_index].split = None
        trees[best_index].left_tree = None
        trees[best_index].right_tree = None
        trees[best_index].is_node = True
        trees[best_index].value = means[best_index]
        trees[best_index].annotations = {}
        
        return True
    
    def annotations_and_means(self, train_features, train_targets):
        if not self.is_node:
            yield self, train_targets.mean(), self.annotations["cost"]
            
            left_split, right_split = decision_split_df(train_features, train_targets, self.split)
            
            for tree, cost, mean in self.left_tree.annotations_and_means(*left_split):
                yield tree, cost, mean
                
            for tree, cost, mean in self.right_tree.annotations_and_means(*right_split):
                yield tree, cost, mean
    
    def paths(self):
        return self._paths([])
    
    def _paths(self, partial_path):
        if self.is_node:
            yield partial_path, self.value
        else:
            if self.left_tree is not None:
                new_path = partial_path + [(self.split, False)]
                for path in self.left_tree._paths(new_path):
                    yield path
            
            if self.right_tree is not None:
                new_path = partial_path + [(self.split, True)]
                for path in self.right_tree._paths(new_path):
                    yield path
                    
    def classify(self, df):
        values = pd.Series(0., index=df.index)
        
        for path, value in self.paths():
            values[self.where_path_applies(df, path)] = value
        
        return values
    
    def where_path_applies(self, df, path):
        mask = pd.Series(True, index=df.index)
        
        for (split, choose_greater) in path:
            feature, split_value = split
            
            values_greater = df[feature] >= split_value
            
            new_mask = df[feature] > split_value if choose_greater else df[feature] <= split_value
            
            mask = np.logical_and(mask, new_mask)
        
        return mask
    
    def prune_to_smallest_subtree(self, train_features, train_targets, threshold=1e-8):
        self.annotate_errors(train_features, train_targets)
        
        while self._prune_to_smallest_subtree(threshold=threshold):
            pass
    
    def _prune_to_smallest_subtree(self, threshold):
        if not self.is_node:
            if self.left_tree.is_node and self.right_tree.is_node:
                left_error = self.left_tree.annotations["error"]
                right_error = self.right_tree.annotations["error"]
                
                if (self.annotations["error"] - (left_error + right_error)) < threshold:
                    self.left_tree = None
                    self.right_tree = None
                    self.split = None
                    self.is_node = True
                    self.value = self.annotations["mean"]
                    
                    return True
                
                return False
            else:
                left_was_pruned = self.left_tree._prune_to_smallest_subtree(threshold)
                right_was_pruned = self.right_tree._prune_to_smallest_subtree(threshold)
                
                return left_was_pruned or right_was_pruned
        else:
            return False
    
    def prune_weakest_link(self, train_features, train_targets):
        if self.is_node:
            return 0., 0
        
        self.annotate_branch_errors(train_features, train_targets)
        
        trees, branch_errors, leaf_counts, errors = zip(*list(self.branch_errors()))
        
        branch_errors = np.array(branch_errors)
        leaf_counts = np.array(leaf_counts)
        errors = np.array(errors)
        
        alphas = (errors - branch_errors) / (leaf_counts - 1)
        
        weakest_alpha = np.min(alphas)
        
        pruneables = np.nonzero(alphas == weakest_alpha)
        
        for prune_index in pruneables:
            prune_index = prune_index[0]
            trees[prune_index].split = None
            trees[prune_index].left_tree = None
            trees[prune_index].right_tree = None
            trees[prune_index].is_node = True
            trees[prune_index].value = trees[prune_index].annotations["mean"]
            
        return weakest_alpha, len(pruneables)

    def branch_errors(self):
        if not self.is_node:
            yield self, self.annotations["branch_error"], self.annotations["leaf_count"], self.annotations["error"]
            
            for tree, branch_error, leaf_count, error in self.left_tree.branch_errors():
                yield tree, branch_error, leaf_count, error
                
            for tree, branch_error, leaf_count, error in self.right_tree.branch_errors():
                yield tree, branch_error, leaf_count, error
        
    def annotate_errors(self, train_features, train_targets):
        self.annotations["error"] = ((train_targets.mean() - train_targets)**2).sum()
        self.annotations["mean"] = train_targets.mean()
        
        if not self.is_node:
            left_split, right_split = decision_split_df(train_features, train_targets, self.split)

            self.left_tree.annotate_errors(*left_split)
            self.right_tree.annotate_errors(*right_split)
            
    def annotate_branch_errors(self, train_features, train_targets):
        if self.is_node:
            return rss_from_mean(train_targets), 1
        else:
            left_split, right_split = decision_split_df(train_features, train_targets, self.split)
            
            left_branch_error, left_leaf_count = self.left_tree.annotate_branch_errors(*left_split)
            right_branch_error, right_leaf_count = self.right_tree.annotate_branch_errors(*right_split)
            
            self.annotations["branch_error"] = left_branch_error + right_branch_error
            self.annotations["leaf_count"] = left_leaf_count + right_leaf_count
            
            return self.annotations["branch_error"], self.annotations["leaf_count"]
            
    def node_count(self):
        if self.is_node:
            return 1
        
        return 1 + self.left_tree.node_count() + self.right_tree.node_count()


def make_stump(split, left_value, right_value):
    return DecisionTree(split=split, left_tree=DecisionTree(value=left_value), right_tree=DecisionTree(value=right_value))

def make_node(value):
    return DecisionTree(value=value)

In [3]:
feature_df = pd.read_hdf(dirs.processed / "book_features.h5")
targets_df = pd.read_hdf(dirs.processed / "targets_train.h5")

In [4]:
feature_val = feature_df.sample(frac=0.2, random_state=5).sort_index()
val_index = feature_val.index

feature_train = feature_df.drop(val_index).sort_index()

In [5]:
def make_split(df):
    return df.loc[feature_train.index], df.loc[feature_val.index]

In [6]:
targets_train, targets_val = make_split(targets_df)

In [7]:
feature_train

past_vol  wap1_mean  wap1_std  wap1_min  wap1_med  wap1_max  \
stock_id time_id                                                                
0        5        0.004499   1.003725  0.000693  1.001434  1.003923  1.004920   
         11       0.001204   1.000239  0.000262  0.999700  1.000232  1.000834   
         16       0.002369   0.999542  0.000864  0.997224  0.999818  1.000878   
         62       0.001894   0.999619  0.000258  0.999231  0.999586  1.000159   
         97       0.010034   0.996629  0.001862  0.993168  0.996739  1.000231   
...                    ...        ...       ...       ...       ...       ...   
126      32750    0.003293   1.000025  0.000971  0.998281  0.999979  1.002136   
         32751    0.003691   0.999582  0.000486  0.998250  0.999611  1.000736   
         32753    0.004104   1.002476  0.001264  1.000633  1.002376  1.006166   
         32763    0.003661   1.001809  0.000456  1.000562  1.001788  1.002963   
         32767    0.002092   1.000272  0.000384  0.999178  1.000314  1.001058   

                  wap2_mean  wap2_std  wap2_min  wap2_med  wap2_max  \
stock_id time_id                                                      
0        5         1.003661  0.000781  1.001390  1.003821  1.005124   
         11        1.000206  0.000272  0.999575  1.000192  1.001067   
         16        0.999680  0.000862  0.996897  0.999751  1.000876   
         62        0.999626  0.000317  0.999102  0.999598  1.000249   
         97        0.996725  0.001979  0.992609  0.996681  1.000965   
...                     ...       ...       ...       ...       ...   
126      32750     0.999987  0.001037  0.998192  0.999978  1.002215   
         32751     0.999585  0.000613  0.997950  0.999506  1.000788   
         32753     1.002602  0.001303  1.000632  1.002726  1.006178   
         32763     1.001790  0.000507  1.000537  1.001814  1.003009   
         32767     1.000367  0.000465  0.999129  1.000457  1.001306   

                  spread_mean  spread_std  spread_min  spread_med  spread_max  
stock_id time_id                                                               
0        5           0.000852    0.000212    0.000361    0.000876    0.001394  
         11          0.000394    0.000157    0.000151    0.000351    0.000904  
         16          0.000725    0.000164    0.000384    0.000718    0.001150  
         62          0.000397    0.000130    0.000093    0.000373    0.000793  
         97          0.001666    0.000446    0.000459    0.001735    0.002663  
...                       ...         ...         ...         ...         ...  
126      32750       0.000774    0.000214    0.000206    0.000786    0.001200  
         32751       0.000878    0.000235    0.000392    0.000882    0.001570  
         32753       0.000706    0.000228    0.000240    0.000688    0.001372  
         32763       0.000530    0.000172    0.000066    0.000526    0.001117  
         32767       0.000432    0.000125    0.000154    0.000412    0.000823  

[225499 rows x 16 columns]

In [8]:
tree = DecisionTree(split=("past_vol", 0.003), left_tree=make_stump(("wap1_mean", 1.), 0.0010, 0.0020), right_tree=DecisionTree(value=0.0045))

list(tree.paths())

[([(('past_vol', 0.003), False), (('wap1_mean', 1.0), False)], 0.001),
 ([(('past_vol', 0.003), False), (('wap1_mean', 1.0), True)], 0.002),
 ([(('past_vol', 0.003), True)], 0.0045)]

In [9]:
rmspe(tree.classify(feature_train), targets_train)

0.4755109243894239

In [10]:
def fit_tree(train_features, train_targets, min_observations=1, tqdm_iter=None):
    if len(train_features) <= min_observations:
        if len(train_features) == 0:
            return make_node(0.)
        
        return make_node(train_targets.mean())
    
    # s_scores = np.zeros(len(train_features.columns), dtype=float)
    # lits = np.zeros(len(train_features.columns), dtype=float
    features = np.array(train_features.columns)
    
    best_splits = np.array([get_best_split(train_features[feature], train_targets) for feature in features])
    
    best_index = trial_splits(train_features.to_numpy(), train_targets.to_numpy(), best_splits)
    
#     for index, feature in enumerate(features):
#         best_split = get_best_split(train_features[feature], train_targets)
        
#         less_mean = train_targets[train_features[feature] <= best_split].mean()
#         greater_mean = train_targets[train_features[feature] > best_split].mean()
        
#         best_stump = make_stump((feature, best_split), less_mean, greater_mean)
        
#         rss_scores[index] = np.sum((best_stump.classify(train_features) - train_targets)**2)
#         splits[index] = best_split
    
#     best_index = np.argmin(rss_scores)
    
    left_bool_index = train_features[features[best_index]] <= best_splits[best_index]
    right_bool_index = train_features[features[best_index]] > best_splits[best_index]
    
    if len(train_features[left_bool_index]) == 0 or len(train_features[right_bool_index]) == 0:
        return make_node(train_targets.mean())
    
    left_rss = rss_from_mean(train_targets[left_bool_index])
    right_rss = rss_from_mean(train_targets[right_bool_index])
    
    if (left_rss + right_rss) > rss_from_mean(train_targets):
        return make_node(train_targets.mean())
    
    if tqdm_iter is not None:
        tqdm_iter.update(1)
    
    left_tree = fit_tree(train_features[left_bool_index], train_targets[left_bool_index], min_observations, tqdm_iter)
    right_tree = fit_tree(train_features[right_bool_index], train_targets[right_bool_index], min_observations, tqdm_iter)
    
    return DecisionTree(split=(features[best_index], best_splits[best_index]), left_tree=left_tree, right_tree=right_tree)


def get_best_split(train_feature, train_targets):
    train_feature_np = train_feature.to_numpy()
    train_targets_np = train_targets.to_numpy()
    
    splits = np.copy(train_targets_np)
    
    best_index = trial_splits(train_feature_np[:, None], train_targets_np, splits)
    
    return splits[best_index]


def trial_splits(train_feature, train_targets, splits):
    greater_matrix = train_feature > splits[None, :]
    less_matrix = np.logical_not(greater_matrix)
    
    greater_matrix_sum = np.sum(greater_matrix, axis=0)
    less_matrix_sum = np.sum(less_matrix, axis=0)
    
    greater_matrix_sum[greater_matrix_sum == 0] = 1
    less_matrix_sum[less_matrix_sum == 0] = 1

    greater_means = np.sum(train_targets[:, None] * greater_matrix, axis=0) / greater_matrix_sum
    less_means = np.sum(train_targets[:, None] * less_matrix, axis=0) / less_matrix_sum
    
    greater_diffs = (greater_means[None, :] - train_targets[:, None]) * greater_matrix
    less_diffs = (less_means[None, :] - train_targets[:, None]) * less_matrix
    
    rss_scores = np.sum(greater_diffs**2, axis=0) + np.sum(less_diffs**2, axis=0)
    
    return np.argmin(rss_scores)

In [11]:
len(feature_train.loc[110])

2422

In [12]:
stock_id = 110

train_x, train_y = feature_train.loc[stock_id], targets_train.loc[stock_id]
val_x, val_y = feature_val.loc[stock_id], targets_val.loc[stock_id]

with tqdm(leave=True) as pbar:
    overfit_tree = fit_tree(feature_train.loc[stock_id], targets_train.loc[stock_id], min_observations=1, tqdm_iter=pbar)

0it [00:00, ?it/s]

In [13]:
overfit_tree.node_count()

769

In [14]:
rmspe(overfit_tree.classify(feature_train.loc[stock_id]), targets_train.loc[stock_id])

0.3031509888641468

In [15]:
def annotate_cost(decision_tree, train_features, train_targets, tuning_param=0.):
    if not decision_tree.is_node:
        (train_features_left, train_targets_left), (train_features_right, train_targets_right) = decision_split_df(
            train_features,
            train_targets,
            decision_tree.split
        )
        
        left_cost = annotate_cost(decision_tree.left_tree, train_features_left, train_targets_left, tuning_param=tuning_param)
        right_cost = annotate_cost(decision_tree.right_tree, train_features_right, train_targets_right, tuning_param=tuning_param)
        
        terminal_cost = left_cost + right_cost + tuning_param * decision_tree.annotations["node_count"]
        
        as_node_cost = ((train_targets - train_targets.mean())**2).sum()
        
        decision_tree.annotations["cost"] = terminal_cost - (as_node_cost + tuning_param)
        
        return left_cost + right_cost
    else:
        return ((train_targets - train_targets.mean())**2).sum()


def annotate_node_counts(decision_tree):
    if not decision_tree.is_node:
        left_nodes = annotate_node_counts(decision_tree.left_tree)
        right_nodes = annotate_node_counts(decision_tree.right_tree)

        decision_tree.annotations["node_count"] = left_nodes + right_nodes

        return decision_tree.annotations["node_count"]
    else:
        return 1

In [16]:
# annotate_node_counts(overfit_tree)

# tune_param = 1e-6
# annotate_cost(overfit_tree, feature_train.loc[stock_id], targets_train.loc[stock_id], tuning_param=tune_param)

# ((overfit_tree.classify(feature_train.loc[stock_id]) - targets_train.loc[stock_id])**2).sum()

In [66]:
test = overfit_tree.clone()

In [67]:
test.annotate_errors(train_x, train_y)

In [68]:
test.annotate_branch_errors(train_x, train_y)

(0.0037236206849737405, 385)

In [69]:
test.annotations["error"], test.annotations["branch_error"]

(0.01780801951103141, 0.0037236206849737405)

In [70]:
# ((test.annotations["mean"] - train_y)**2).sum()
((train_y.mean() - train_y)**2).sum()

0.01780801951103141

In [71]:
train_y.mean()

0.004465656952931462

In [72]:
test.left_tree.annotations["error"]

0.005752626179994373

In [73]:
test.prune_to_smallest_subtree(feature_train.loc[stock_id], targets_train.loc[stock_id], threshold=1e-16)

In [74]:
test.node_count()

769

In [75]:
new_alpha, pruned_count = test.prune_weakest_link(feature_train.loc[stock_id], targets_train.loc[stock_id])

with tqdm() as pbar:
    while pruned_count != 0:
        _, pruned_count = test.prune_weakest_link(train_x, train_y)
        pbar.update()

0it [00:00, ?it/s]

In [27]:
# np.sort(alphas)

In [76]:
test.node_count()

1

In [29]:
# annotate_node_counts(test)
# annotate_cost(test, feature_train.loc[stock_id], targets_train.loc[stock_id], tuning_param=tune_param)
# prune_more = test.prune_best_cost(feature_train.loc[stock_id], targets_train.loc[stock_id])

# with tqdm() as pbar:
#     while prune_more:
#         annotate_node_counts(test)
#         annotate_cost(test, feature_train.loc[stock_id], targets_train.loc[stock_id], tuning_param=tune_param)
#         prune_more = test.prune_best_cost(feature_train.loc[stock_id], targets_train.loc[stock_id])
#         pbar.update()

In [30]:
# annotate_node_counts(test)

In [31]:
rmspe(test.classify(feature_train.loc[stock_id]), targets_train.loc[stock_id])

0.3031509037542305